In [101]:
# import numpy as np
import tensorflow as tf

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, SimpleRNN, TimeDistributed, LSTM, GRU, Bidirectional
from tensorflow.keras.optimizers import SGD, Adam

# 데이터 샘플 생성

In [134]:
# 임의의 데이터 1개 생성
Sample_x = [ 1,   0,   0, 0, 0, 0] 
Sample_y = [ 1, 0.8, 0.6, 0, 0, 0]

In [140]:
np.array([1,2,3] * 3).reshape(3,3)

array([[1, 2, 3],
       [1, 2, 3],
       [1, 2, 3]])

In [141]:
# 똑같은 데이터 50개로 증폭하여 샘플 생성
sample_size = 50

Train_x = np.array(Sample_x*sample_size).reshape(sample_size, len(Sample_x), 1)
Train_y = np.array(Sample_y*sample_size).reshape(sample_size, len(Sample_y), 1)
# Dimension : 샘플의 수, Time domain의 수, X변수의 수

In [144]:
# 데이터 shape 꼭 확인
print(Train_x.shape)
print(Train_y.shape)

# Sample, Time-steps, Data Dimension

(50, 6, 1)
(50, 6, 1)


In [145]:
len(Sample_x)

6

In [160]:
# 모델 구성
model = Sequential()

# RNN. 현재 RNN은 return_sequences (모든 x time-step에 대하여 output을 냄)
model.add(SimpleRNN(1, activation='tanh', 
                    return_sequences = True,
                    input_shape= ( len(Sample_x), 1,  )
                   ))

model.add(TimeDistributed(Dense(1, activation='linear')))
model.compile(loss = "mse", optimizer='Adam')

In [161]:
# 모델 구조 확인
# Simple_rnn의 Parameter의 수가 3인 이유는 1. coef A, 2. Const, 3. RNN State_const
# TimeDistributed ( Dense ) 의 output shape가 왜 (None, 6, 1) 일까?
model.summary()

Model: "sequential_49"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_51 (SimpleRNN)    (None, 6, 1)              3         
_________________________________________________________________
time_distributed_25 (TimeDis (None, 6, 1)              2         
Total params: 5
Trainable params: 5
Non-trainable params: 0
_________________________________________________________________


In [162]:
# Model fitting
model.fit(Train_x, Train_y, epochs=5, batch_size=1, verbose=True)

Epoch 1/5
50/50 [==============================] - 1s 6ms/step - loss: 0.2198
Epoch 2/5
50/50 [==============================] - 0s 6ms/step - loss: 0.1851
Epoch 3/5
50/50 [==============================] - 0s 6ms/step - loss: 0.1588
Epoch 4/5
50/50 [==============================] - 0s 5ms/step - loss: 0.1332
Epoch 5/5
50/50 [==============================] - 0s 6ms/step - loss: 0.1095


In [163]:
# 데이터 1개로 prediction
# Dimension 헷갈리지 않게 체크
model.predict(
    [
        [[1], [0], [0], [0], [0], [0]]
    ]
)

array([[[0.5325564 ],
        [0.409338  ],
        [0.31561193],
        [0.2412745 ],
        [0.18159515],
        [0.13383628]]], dtype=float32)

In [164]:
model.predict(
    [
        [
            [1], [ 0 ], [0], [0], [0], [0]]
        ,
        [
            [1], [0.3], [0], [0], [0], [0]
        ]
    ]
)

array([[[0.5325564 ],
        [0.409338  ],
        [0.31561193],
        [0.2412745 ],
        [0.18159515],
        [0.13383628]],

       [[0.5325564 ],
        [0.5157076 ],
        [0.39689296],
        [0.3058514 ],
        [0.2334531 ],
        [0.17531697]]], dtype=float32)

# Return_sequence가 없는 RNN 모델

(에러나는 이유?)
 `TimeDistributed` Layer should be passed an `input_shape ` with at least 3 dimensions, received: [None, 1]
 input_shape = 최소 3 dimension
 기대하는 Input shape: [batch_size, Time_domain, 변수의 수]
                       [32,         6,         ,  1]
                       return_sequence = True
                       RNN -> output이 input으로 6개의 T를 -> 6개의 output
                       [None, 6, 1]
 


In [166]:
# 모델 구성
model = Sequential()

# RNN. 현재 RNN은 return_sequences (모든 x time-step에 대하여 output을 냄)
model.add(SimpleRNN(1, activation='tanh', return_sequences = False, input_shape=( len(Sample_x), 1,  )))
model.add(TimeDistributed(Dense(1, activation='linear')))
model.compile(loss = "mse", optimizer='Adam')

ValueError: `TimeDistributed` Layer should be passed an `input_shape ` with at least 3 dimensions, received: [None, 1]

<h3>왜 에러가 나는지에 대한 이해</h3>
<br>
1. return_sequences를 False를 하면, RNN의 최종 Output만 출력<br>
2. TimeDistributed는 이전의 Output sequence가 time-series로 출력될 것을 가정. 따라서 3차원의 output이 나올 것으로 예상<br>
( batch_size (1차원) * 이전 Layer의 output (1차원) * Time-series (1차원) )<br>

In [167]:
# 임의의 데이터 1개 생성
Sample_x = [1, 0, 0, 0, 0, 0]
Sample_y = [0]

# many to one

In [172]:
# 똑같은 데이터 100개로 증폭하여 샘플 생성
sample_size = 100

Train_x = np.array(Sample_x*sample_size).reshape(sample_size, len(Sample_x), 1)
Train_y = np.array(Sample_y*sample_size).reshape(sample_size, len(Sample_y))

In [173]:
print(Train_x.shape)
print(Train_y.shape)

(100, 6, 1)
(100, 1)


In [174]:
# 모델 구성
model = Sequential()

# RNN. 현재 RNN은 return_sequences (모든 x time-step에 대하여 output을 냄)
model.add(SimpleRNN(1, activation='tanh', return_sequences = False, input_shape=( len(Sample_x), 1,  )))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss = "binary_crossentropy", optimizer='Adam')

In [175]:
model.summary()

Model: "sequential_53"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_55 (SimpleRNN)    (None, 1)                 3         
_________________________________________________________________
dense_43 (Dense)             (None, 1)                 2         
Total params: 5
Trainable params: 5
Non-trainable params: 0
_________________________________________________________________


In [176]:
# Model fitting
model.fit(Train_x, Train_y, epochs=5, batch_size=1, verbose=True)

Epoch 1/5
100/100 [==============================] - 1s 6ms/step - loss: 0.4923
Epoch 2/5
100/100 [==============================] - 1s 6ms/step - loss: 0.3912
Epoch 3/5
100/100 [==============================] - 1s 6ms/step - loss: 0.3165
Epoch 4/5
100/100 [==============================] - 1s 6ms/step - loss: 0.2615
Epoch 5/5
100/100 [==============================] - 1s 6ms/step - loss: 0.2198


# Test 예측하기

In [177]:
# 데이터 1개로 prediction
# Dimension 헷갈리지 않게 체크, output은 1개
model.predict(
    [
        [[1], [0], [0], [0], [0], [0]]
    ]
)

array([[0.17692749]], dtype=float32)

# 모델 구성도 상상해보기

In [130]:
# 모델 구성
model = Sequential()

# RNN. 현재 RNN은 return_sequences (모든 x time-step에 대하여 output을 냄)
model.add(SimpleRNN(1, activation='tanh', return_sequences = True, input_shape=( len(Sample_x), 1,  )))
model.add(SimpleRNN(1, activation='tanh', return_sequences = False))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss = "binary_crossentropy", optimizer='Adam')

model.summary()

Model: "sequential_41"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_40 (SimpleRNN)    (None, 6, 1)              3         
_________________________________________________________________
simple_rnn_41 (SimpleRNN)    (None, 1)                 3         
_________________________________________________________________
dense_31 (Dense)             (None, 1)                 2         
Total params: 8
Trainable params: 8
Non-trainable params: 0
_________________________________________________________________


In [131]:
# 모델 구성
model = Sequential()

# RNN. 현재 RNN은 return_sequences (모든 x time-step에 대하여 output을 냄)
model.add(SimpleRNN(1, activation='tanh', return_sequences = True, input_shape=( len(Sample_x), 1,  )))
model.add(SimpleRNN(1, activation='tanh', return_sequences = True))
model.add(TimeDistributed(Dense(1, activation='linear')))
model.compile(loss = "binary_crossentropy", optimizer='Adam')

model.summary()

Model: "sequential_42"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_42 (SimpleRNN)    (None, 6, 1)              3         
_________________________________________________________________
simple_rnn_43 (SimpleRNN)    (None, 6, 1)              3         
_________________________________________________________________
time_distributed_20 (TimeDis (None, 6, 1)              2         
Total params: 8
Trainable params: 8
Non-trainable params: 0
_________________________________________________________________


In [132]:
# 모델 구성
model = Sequential()

# RNN. 현재 RNN은 return_sequences (모든 x time-step에 대하여 output을 냄)
model.add(SimpleRNN(1, activation='tanh', return_sequences = True, input_shape=( len(Sample_x), 1,  )))
model.add(SimpleRNN(1, activation='tanh', return_sequences = True))
model.add(TimeDistributed(Dense(1, activation='linear')))
model.compile(loss = "binary_crossentropy", optimizer='Adam')

model.summary()

Model: "sequential_43"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_44 (SimpleRNN)    (None, 6, 1)              3         
_________________________________________________________________
simple_rnn_45 (SimpleRNN)    (None, 6, 1)              3         
_________________________________________________________________
time_distributed_21 (TimeDis (None, 6, 1)              2         
Total params: 8
Trainable params: 8
Non-trainable params: 0
_________________________________________________________________


In [133]:
model = Sequential()
model.add(SimpleRNN(5, return_sequences=True, input_shape=( len(Sample_x), 1,  )))
model.add(Bidirectional(GRU(5, return_sequences=True)))
model.add(Bidirectional(LSTM(5, return_sequences=True)))
model.add(TimeDistributed(Dense(1, activation="relu")))
model.compile(loss = "binary_crossentropy", optimizer='Adam')
model.summary()

Model: "sequential_44"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_46 (SimpleRNN)    (None, 6, 5)              35        
_________________________________________________________________
bidirectional_12 (Bidirectio (None, 6, 10)             360       
_________________________________________________________________
bidirectional_13 (Bidirectio (None, 6, 10)             640       
_________________________________________________________________
time_distributed_22 (TimeDis (None, 6, 1)              11        
Total params: 1,046
Trainable params: 1,046
Non-trainable params: 0
_________________________________________________________________
